In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [33]:
!pip install beautifulsoup4


In [29]:
from bs4 import BeautifulSoup

def fetch_recipe(dish):
    api_key = "5a92be6a1d0c411197ac5692c09d6a2f"  # Replace with your valid API key
    search_url = f"https://api.spoonacular.com/recipes/complexSearch?query={dish}&apiKey={api_key}&number=1"
    try:
        response = requests.get(search_url).json()
        if 'results' in response and response['results']:
            recipe_id = response['results'][0]['id']
            recipe_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={api_key}&includeNutrition=false"
            details = requests.get(recipe_url).json()
            ingredients = details.get('extendedIngredients', [])
            instructions_html = details.get('instructions', 'No instructions found.')

            # Clean HTML using BeautifulSoup
            soup = BeautifulSoup(instructions_html, "html.parser")
            instructions = soup.get_text()

            image_url = details.get('image', f"https://via.placeholder.com/400x400.png?text={dish.replace(' ', '+')}")
            ingredient_list = [f"{item['original']}" for item in ingredients]
            return ingredient_list, instructions, image_url
        else:
            return None, f"No recipes found for the dish: {dish}.", None
    except Exception as e:
        return None, f"Error fetching recipe: {str(e)}", None


In [30]:
def generate_recipe(dish):
    ingredients, instructions, image_url = fetch_recipe(dish)
    if not ingredients:
        return "Recipe not found!", None, None

    # Create HTML for ingredients and steps
    ingredient_text = "<br>".join([f"• {item}" for item in ingredients])
    full_recipe_html = f"""
    <h3>Ingredients:</h3>
    <p>{ingredient_text}</p>
    <h3>Steps:</h3>
    {instructions}  <!-- This keeps the HTML formatting -->
    """

    # Generate and display the image
    response = requests.get(image_url)
    generated_image = Image.open(BytesIO(response.content))

    # Create YouTube link
    youtube_link = f"https://www.youtube.com/results?search_query={dish}+recipe"
    youtube_html = f'<a href="{youtube_link}" target="_blank">Watch Recipe on YouTube</a>'

    return full_recipe_html, generated_image, youtube_html


In [26]:
gr.Interface(
    fn=generate_recipe,
    inputs=gr.Textbox(placeholder="Enter dish name", label="Dish Name"),
    outputs=[
        gr.HTML(label="Recipe Instructions"),
        gr.Image(label="Dish Image"),
        gr.HTML(label="YouTube Link")
    ]
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c8428ba611ab7a5a7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Similar recipe Recommendations

In [34]:
def fetch_similar_recipes(recipe_id, api_key):
    similar_url = f"https://api.spoonacular.com/recipes/{recipe_id}/similar?apiKey={api_key}&number=3"
    try:
        response = requests.get(similar_url).json()
        similar_recipes = [f"{item['title']} - {item['sourceUrl']}" for item in response]
        return similar_recipes
    except Exception as e:
        return [f"Error fetching similar recipes: {str(e)}"]


In [35]:
def fetch_recipe(dish):
    api_key = "5a92be6a1d0c411197ac5692c09d6a2f"  # Replace with your Spoonacular API key
    search_url = f"https://api.spoonacular.com/recipes/complexSearch?query={dish}&apiKey={api_key}&number=1"
    try:
        response = requests.get(search_url).json()
        if 'results' in response and response['results']:
            recipe_id = response['results'][0]['id']
            recipe_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={api_key}&includeNutrition=false"
            details = requests.get(recipe_url).json()
            ingredients = details.get('extendedIngredients', [])
            instructions_html = details.get('instructions', 'No instructions found.')

            # Clean HTML using BeautifulSoup
            soup = BeautifulSoup(instructions_html, "html.parser")
            instructions = soup.get_text()

            image_url = details.get('image', f"https://via.placeholder.com/400x400.png?text={dish.replace(' ', '+')}")
            ingredient_list = [f"{item['original']}" for item in ingredients]

            # Fetch similar recipes
            similar_recipes = fetch_similar_recipes(recipe_id, api_key)

            return ingredient_list, instructions, image_url, similar_recipes
        else:
            return None, f"No recipes found for the dish: {dish}.", None, []
    except Exception as e:
        return None, f"Error fetching recipe: {str(e)}", None, []


Speech synthesis

In [36]:
!pip install gtts


In [37]:
from gtts import gTTS
import os

def text_to_speech(text, output_file="recipe_audio.mp3"):
    try:
        tts = gTTS(text)
        tts.save(output_file)
        return output_file
    except Exception as e:
        return f"Error generating speech: {str(e)}"


In [38]:
def generate_recipe(dish):
    ingredients, instructions, image_url, similar_recipes = fetch_recipe(dish)
    if not ingredients:
        return "Recipe not found!", None, None, []

    # Create text for TTS
    tts_text = f"Here is the recipe for {dish}. Ingredients: " + ", ".join(ingredients) + ". Steps: " + instructions

    # Generate TTS
    tts_file = text_to_speech(tts_text)

    # Create HTML for similar recipes
    similar_recipes_html = "<br>".join([f"• {recipe}" for recipe in similar_recipes])

    # Generate and display the image
    response = requests.get(image_url)
    generated_image = Image.open(BytesIO(response.content))

    # Create YouTube link
    youtube_link = f"https://www.youtube.com/results?search_query={dish}+recipe"

    return (
        f"<h3>Ingredients:</h3><p>{'<br>'.join(ingredients)}</p><h3>Steps:</h3>{instructions}",
        generated_image,
        f'<a href="{youtube_link}" target="_blank">Watch Recipe on YouTube</a>',
        f"<h3>Similar Recipes:</h3><p>{similar_recipes_html}</p>",
        tts_file,
    )


In [40]:
gr.Interface(
    fn=generate_recipe,
    inputs=gr.Textbox(placeholder="Enter dish name", label="Dish Name"),
    outputs=[
        gr.HTML(label="Recipe Instructions"),
        gr.Image(label="Dish Image"),
        gr.HTML(label="YouTube Link"),
        gr.HTML(label="Similar Recipes"),
        gr.Audio(label="Recipe Audio"),
    ],
    live=True
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d927349d77c35d2bab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
